# Initialization

In [1]:
%run 050-init.ipynb

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 17.0.8
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.
--2023-08-29 17:06:50--  https://raw.githubusercontent.com/meddash-cloud/meddash-public-datasets/main/data/movielens_ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14373 (14K) [text/plain]
Saving to: ‘movielens_ratings.csv.1’

movielens_ratings.c 100%[===================>]  14.04K  --.-KB/s    in 0s      

2023-08-29 17:06:51 (41.6 MB/s

# Recommender Code

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [2]:
from pyspark.sql import SparkSession

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [5]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [6]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [7]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [8]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [9]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      7|   1.0|    28|    2.163055|
|      1|   1.0|     6|    4.127722|
|      0|   1.0|     3|   0.1507456|
|      1|   1.0|    20|   1.2038493|
|      1|   1.0|     5|  0.81468815|
|      0|   1.0|    19|  0.96844816|
|      4|   1.0|    19|   3.2358496|
|      2|   3.0|     9|   1.1974491|
|      4|   1.0|     9|  -1.0301473|
|      3|   2.0|     8|  0.20560317|
|      5|   1.0|     8|   1.8034872|
|      4|   1.0|     7|   1.6191181|
|      2|   4.0|    10|   2.2984483|
|      7|   1.0|    25|   1.5059602|
|      2|   4.0|    21|   1.4684672|
|      0|   1.0|    11|   2.1307366|
|      5|   1.0|    14|        1.33|
|      6|   1.0|    14|   2.1216571|
|      4|   3.0|     2|  -0.7349599|
|      3|   1.0|     0|0.0011359835|
+-------+------+------+------------+
only showing top 20 rows



In [10]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8454261954461646


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [11]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [12]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|      9|    11|
|     10|    11|
|     22|    11|
|     36|    11|
|     40|    11|
|     50|    11|
|     61|    11|
|     62|    11|
|     67|    11|
|     72|    11|
|     78|    11|
|     79|    11|
|     81|    11|
|     90|    11|
+-------+------+



In [13]:
reccomendations = model.transform(single_user)

In [14]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     50|    11|   3.2936885|
|     22|    11|   3.2028222|
|      0|    11|   2.1307366|
|     10|    11|   2.1152635|
|     67|    11|   1.1972272|
|     79|    11|    1.124777|
|     90|    11|   0.5516186|
|     36|    11|  0.34837383|
|     61|    11|  0.16516136|
|     40|    11| 0.110410266|
|     81|    11|-0.040581286|
|     78|    11|-0.041135956|
|      9|    11|  -1.8340907|
|     72|    11|  -2.3804708|
|     62|    11|  -2.7856727|
+-------+------+------------+

